In [2]:
import setup_django
setup_django.init()
print("hello")

hello


In [9]:
from movies.models import Movie

qs = Movie.objects.all().values('title','release_date', 'rating_avg', 'rating_count')
qs

<MovieQuerySet [{'title': 'Ariel', 'release_date': datetime.date(1988, 10, 21), 'rating_avg': Decimal('2.80'), 'rating_count': 5}, {'title': 'Shadows in Paradise', 'release_date': datetime.date(1986, 10, 16), 'rating_avg': Decimal('3.60'), 'rating_count': 5}, {'title': 'Four Rooms', 'release_date': datetime.date(1995, 12, 9), 'rating_avg': Decimal('2.00'), 'rating_count': 7}, {'title': 'Judgment Night', 'release_date': datetime.date(1993, 10, 15), 'rating_avg': Decimal('2.86'), 'rating_count': 7}, {'title': 'Star Wars', 'release_date': datetime.date(1977, 5, 25), 'rating_avg': Decimal('3.42'), 'rating_count': 12}, {'title': 'Finding Nemo', 'release_date': datetime.date(2003, 5, 30), 'rating_avg': Decimal('2.82'), 'rating_count': 11}, {'title': 'Forrest Gump', 'release_date': datetime.date(1994, 7, 6), 'rating_avg': Decimal('2.55'), 'rating_count': 11}, {'title': 'American Beauty', 'release_date': datetime.date(1999, 9, 15), 'rating_avg': Decimal('2.60'), 'rating_count': 10}, {'title': 

In [7]:
import pandas as pd
from django.conf import settings

ratings_path = settings.DATA_DIR / "ratings_small.csv"
ratings_path.exists()

True

In [9]:
df = pd.read_csv(ratings_path)

In [10]:
df.head(n=10)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [12]:
from django.contrib.auth import get_user_model
User = get_user_model()

In [22]:
current_users = User.objects.all().values_list('id', flat=True)
rating_users = df['userId'].tolist()

In [23]:
missing_user_ids = set(rating_users) - set(current_users) 
missing_user_ids

set()

In [20]:
for uid in missing_user_ids:
    User.objects.create(
        id=uid,
        username=f"missing-user-{uid}"
    )

IntegrityError: UNIQUE constraint failed: auth_user.id

In [30]:
import math
from decimal import Decimal

In [31]:
df['value'] = df['rating'].apply(lambda x: math.ceil(Decimal(x)))
df.head()

,userId,movieId,rating,timestamp,value
0,1,31,2.5,1260759144,3
1,1,1029,3.0,1260759179,3
2,1,1061,3.0,1260759182,3
3,1,1129,2.0,1260759185,2
4,1,1172,4.0,1260759205,4


In [34]:
df['user_id'] = df['userId']
df['object_id'] = df['movieId']

In [35]:
df.head()

,userId,movieId,rating,timestamp,value,user_id,object_id
0,1,31,2.5,1260759144,3,1,31
1,1,1029,3.0,1260759179,3,1,1029
2,1,1061,3.0,1260759182,3,1,1061
3,1,1129,2.0,1260759185,2,1,1129
4,1,1172,4.0,1260759205,4,1,1172


In [36]:
cols = ['user_id','value','object_id']
transformed_df = df.copy()[cols]

In [39]:
rating_records = transformed_df.to_dict('records')

In [41]:
from ratings.models import Rating
qs = Rating.objects.all()
qs.delete()

(0, {})

In [43]:
from django.contrib.contenttypes.models import ContentType
ctype = ContentType.objects.get(app_label='movies', model='movie')


In [45]:
new_ratings = []
for r in rating_records:
    r['content_type'] = ctype
    new_ratings.append(
        Rating(**r)
    )
Rating.objects.bulk_create(new_ratings, ignore_conflicts=True, batch_size=1000)

[<Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: 

In [49]:
from ratings.tasks import task_update_movie_ratings
task_update_movie_ratings()

Rating update took: 0:00:10(10.57360577583313s)
